In [ ]:
import pandas as pd
import re
import pdfplumber
from datetime import datetime
import os

In [ ]:
def extract_amex_invoice_data(pdf_path):
    """
    Extract transaction data from American Express PDF invoice
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        tuple: (invoice_period, primary_card_df, extra_card_df)
    """
    # Extract text from PDF
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        # Process all pages
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"
    
    # Extract invoice period
    invoice_period_match = re.search(r'Fakturans period: (\d{2}\.\d{2}\.\d{2}) till (\d{2}\.\d{2}\.\d{2})', all_text)
    if invoice_period_match:
        start_date = invoice_period_match.group(1)
        end_date = invoice_period_match.group(2)
        invoice_period = f"{start_date} - {end_date}"
    else:
        invoice_period = "Unknown"
    
    # Identify transaction sections
    # Find primary cardholder transactions
    primary_start = all_text.find("Nya köp för Roshan Talimi")
    primary_end = all_text.find("Summa nya köp för Roshan Talimi")
    
    # Find extra cardholder transactions
    extra_start = all_text.find("Nya köp för Stephanie Maria Gardner")
    extra_end = all_text.find("Summa nya köp för Stephanie Maria Gardner")
    
    # Extract transaction blocks
    primary_transactions_text = all_text[primary_start:primary_end]
    extra_transactions_text = all_text[extra_start:extra_end]
    
    # Parse transactions
    primary_df = parse_transactions(primary_transactions_text)
    extra_df = parse_transactions(extra_transactions_text)
    
    return invoice_period, primary_df, extra_df

def parse_transactions(transactions_text):
    """
    Parse transaction text block into structured data
    
    Args:
        transactions_text (str): Text containing transactions
        
    Returns:
        pandas.DataFrame: DataFrame with transaction data
    """
    # Regular expression to match transaction lines
    # Format: date date description amount
    transaction_pattern = r'(\d{2}\.\d{2}\.\d{2})\s+(\d{2}\.\d{2}\.\d{2})\s+(.*?)\s+(-?\d+\.\d{2}|-?\d+,\d{2}|-?\d+)$'
    
    # Find all transactions
    transactions = []
    
    for line in transactions_text.split('\n'):
        match = re.search(transaction_pattern, line.strip())
        if match:
            trans_date = match.group(1)
            process_date = match.group(2)
            
            # Get description and amount
            rest_of_line = match.group(3) + ' ' + match.group(4)
            
            # Split the rest by finding the last number which is the amount
            amount_pattern = r'(-?\d+\.\d{2}|-?\d+,\d{2}|-?\d+)$'
            amount_match = re.search(amount_pattern, rest_of_line)
            
            if amount_match:
                amount = amount_match.group(1).replace(',', '.')
                description = rest_of_line[:amount_match.start()].strip()
                
                transactions.append({
                    'Transaction Date': trans_date,
                    'Process Date': process_date,
                    'Description': description,
                    'Amount': float(amount)
                })
    
    # Create DataFrame
    df = pd.DataFrame(transactions)
    
    # Convert dates to datetime objects
    if not df.empty:
        for date_col in ['Transaction Date', 'Process Date']:
            df[date_col] = pd.to_datetime(df[date_col], format='%d.%m.%y')
    
    return df

def process_monthly_invoice(pdf_path, output_dir=None):
    """
    Process a monthly American Express invoice and save the results
    
    Args:
        pdf_path (str): Path to the PDF invoice
        output_dir (str, optional): Directory to save the output CSV files
        
    Returns:
        tuple: (invoice_period, primary_card_df, extra_card_df)
    """
    invoice_period, primary_df, extra_df = extract_amex_invoice_data(pdf_path)
    
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        period_for_filename = invoice_period.replace(' - ', '_to_').replace('.', '')
        
        primary_df.to_csv(os.path.join(output_dir, f"primary_card_{period_for_filename}.csv"), index=False)
        extra_df.to_csv(os.path.join(output_dir, f"extra_card_{period_for_filename}.csv"), index=False)
        
        print(f"Saved transaction data for period {invoice_period}")
        print(f"Primary card transactions: {len(primary_df)}")
        print(f"Extra card transactions: {len(extra_df)}")
    
    return invoice_period, primary_df, extra_df

In [ ]:
pdf_path = "../amex/data/original/2025-02-27.pdf"  # Path to your AmEx PDF invoice
output_dir = "../amex/data/output"  # Directory to save the extracted data

invoice_period, primary_df, extra_df = process_monthly_invoice(pdf_path, output_dir)

# Display the first few rows of each DataFrame
print("\nPrimary Card Transactions:")
print(primary_df.head())

print("\nExtra Card Transactions:")
print(extra_df.head())

In [ ]:
# Example usage
if __name__ == "__main__":
    pdf_path = "2025-02-27.pdf"  # Path to your AmEx PDF invoice
    output_dir = "amex_transactions"  # Directory to save the extracted data
    
    invoice_period, primary_df, extra_df = process_monthly_invoice(pdf_path, output_dir)
    
    # Display the first few rows of each DataFrame
    print("\nPrimary Card Transactions:")
    print(primary_df.head())
    
    print("\nExtra Card Transactions:")
    print(extra_df.head())